In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as wg
from IPython.display import display

time_ticks = 601  # time points to plot
time_final = 600.0  # final time

def process_heater(y, t, u):
    dydt = (23.0 - y + 0.9 * u) / 175
    return dydt


def pidPlot(coef_P, coef_I, coef_D):
    time_vector = np.linspace(0, time_final, time_ticks)  # create time vector
    term_proportional = np.zeros(time_ticks)  # initialize proportional term
    term_integral = np.zeros(time_ticks)  # initialize integral term
    term_differential = np.zeros(time_ticks)  # initialize derivative term

    error = np.zeros(time_ticks)  # initialize error
    controller_output = np.zeros(time_ticks)  # initialize controller output
    process_value = np.ones(time_ticks) * 23.0  # initialize process variable

    set_point = np.ones(time_ticks) * 23.0  # initialize setpoint
    set_point[10:300] = 50.0  # step up
    set_point[300:601] = 40.0  # step down

    y0 = 23.0  # initial condition
    iae = 0.0  # accumulated error

    # loop through all time steps
    for i in range(1, time_ticks):
        # simulate process for one time step
        time_interval = [time_vector[i - 1], time_vector[i]]  # time interval
        y = odeint(process_heater, y0, time_interval, args=(controller_output[max(0, i - 15)],))
        y0 = y[1]  # get new initial condition
        iae += np.abs(set_point[i] - y0[0])
        process_value[i] = y0

        error[i] = set_point[i] - process_value[i]  # calculate error
        dt = time_vector[i] - time_vector[i - 1]  # calculate time step

        term_proportional[i] = coef_P * error[i]  # calculate proportional term
        term_integral[i] = term_integral[i - 1] + (coef_P / coef_I) * error[i] * dt  # calculate integral term
        term_differential[i] = -coef_P * coef_D * (
                    process_value[i] - process_value[i - 1]) / dt  # calculate derivative term

        controller_value = term_proportional[i] + term_integral[i] + term_differential[i]  # calculate new controller output
        controller_output[i] = max(min(100.0, controller_value), 0.0)

        if controller_value <= 0 or controller_value >= 100:
            term_integral[i] = term_integral[i - 1]  # reset integral

    # plot PID response
    # plot 1
    plt.figure(1, figsize=(15, 7))
    plt.subplot(2, 2, 1)
    plt.plot(time_vector, set_point, 'k-', linewidth=2, label='Setpoint (SP)')
    plt.plot(time_vector, process_value, 'r:', linewidth=2, label='Temperature (PV)')
    plt.ylabel(r'T $(^oC)$')
    plt.text(100, 30, 'Integral Abs Error: ' + str(np.round(iae, 2)))
    plt.text(400, 30, r'$K_c$: ' + str(np.round(coef_P, 0)))
    plt.text(400, 27, r'$\tau_I$: ' + str(np.round(coef_I, 0)) + ' sec')
    plt.text(400, 24, r'$\tau_D$: ' + str(np.round(coef_D, 0)) + ' sec')
    plt.legend(loc='best')

    # plot 2
    plt.subplot(2, 2, 2)
    plt.plot(time_vector, term_proportional, 'g.-', linewidth=2, label=r'Proportional = $K_c \; e(t)$')
    plt.plot(time_vector, term_integral, 'b-', linewidth=2, label=r'Integral = ' + \
                                                                  r'$\frac{K_c}{\tau_I} \int_{i=0}^{n_t} e(t) \; dt $')
    plt.plot(time_vector, term_differential, 'r--', linewidth=2, label=r'Derivative = ' + \
                                                                       r'$-K_c \tau_D \frac{d(PV)}{dt}$')
    plt.legend(loc='best')

    # plot 3
    plt.subplot(2, 2, 3)
    plt.plot(time_vector, error, 'm--', linewidth=2, label='Error (e=SP-PV)')
    plt.ylabel(r'$\Delta T$ $(^oC)$')
    plt.legend(loc='best')

    # plot 4
    plt.subplot(2, 2, 4)
    plt.plot(time_vector, controller_output, 'b--', linewidth=2, label='Heater (OP)')
    plt.legend(loc='best')
    plt.xlabel('time')


Kc_slide = wg.FloatSlider(value=5.0, min=0.0, max=50.0, step=1.0)
tauI_slide = wg.FloatSlider(value=120.0, min=20.0, max=180.0, step=5.0)
tauD_slide = wg.FloatSlider(value=0.0, min=0.0, max=20.0, step=1.0)
wg.interact(pidPlot, coef_P=Kc_slide, coef_I=tauI_slide, coef_D=tauD_slide)
print('PID Simulator: Adjust Kc, tauI, and tauD to achieve lowest Integral Abs Error')

interactive(children=(FloatSlider(value=5.0, description='coef_P', max=50.0, step=1.0), FloatSlider(value=120.…

PID Simulator: Adjust Kc, tauI, and tauD to achieve lowest Integral Abs Error
